In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import subprocess
import numpy as np

In [2]:
def ReadAndFilter(file, files_dir, chrom2use):
    sample = pd.read_csv(files_dir+file, sep="\t")
    sample["VC"] = sample["chrom"].astype(str)+"_"+sample["position"].astype(str)+"_"+sample["ref"].astype(str)+"_"+sample["var"].astype(str)
    sample["T_DP"] = sample["tumor_reads1"]+ sample["tumor_reads2"]
    sample["N_DP"] = sample["normal_reads1"]+sample["normal_reads2"]
    sample["N_VAF"] = sample["normal_var_freq"].str.split("%", expand=True)[0].astype(float)/100
    sample["N_ALT"] = sample["N_VAF"]*sample["N_DP"]
    sample["N_ALT"] = sample["N_ALT"].astype(int)
    sample["T_VAF"] = sample["tumor_var_freq"].str.split("%", expand=True)[0].astype(float)/100
    sample["T_ALT"] = sample["T_VAF"]*sample["T_DP"]
    sample["T_ALT"] = sample["T_ALT"].astype(int)
    sample["chrom"] = sample["chrom"].astype(str)
    sample = sample[sample["chrom"].isin(chrom2use)]
    col2show = ["VC", "N_DP", "N_VAF", "N_ALT", "T_DP", "T_VAF","T_ALT",  "somatic_status", "somatic_p_value"]
    return sample[col2show][(sample["N_VAF"]==0)&(sample["somatic_p_value"]<0.05)#&(sample["T_VAF"]>=0.1)
                            #&(sample["N_DP"]>50)&(sample["T_DP"]>50)&(sample["T_ALT"]>5)
                            &(sample["somatic_status"]=="Somatic")].sort_values(by="somatic_p_value")

In [3]:
chrom2use = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15',
             '16', '17', '18', '19', '20', '21', '22', 'X', 'Y']

In [ ]:
# Re-define FASTQ file directory (i.e., filed_dir)
def SNP(sample_name, chrom2use, files_dir = "/home/bioit/ldschaet/Desktop/FASTQ_tNGS_LA/"):
    snp = ReadAndFilter(sample_name, files_dir = files_dir+"filtered_vcfs/", chrom2use=chrom2use)
    return snp

In [ ]:
# Re-define FASTQ file directory (i.e., filed_dir)
def SNPandIndel(sample_name, chrom2use, files_dir = "/home/bioit/ldschaet/Desktop/FASTQ_tNGS_LA/"):
    snp = ReadAndFilter(sample_name, files_dir = files_dir+"filtered_vcfs/", chrom2use=chrom2use)
    snp["type"] = "snp"
    indel = ReadAndFilter(sample_name+".indel", files_dir = files_dir+"vcfs/", chrom2use=chrom2use)
    indel["type"] = "indel"
    combined = pd.concat([snp, indel])
    return combined

In [6]:
def GetCoverageAtPos(row, bam_file):
    command = ["samtools", "depth", bam_file, "-r", f"{row.chrom}:{row.pos}-{row.pos}"]
    result = subprocess.run(command, stdout=subprocess.PIPE, text=True)
    if result.returncode == 0:
        lines = result.stdout.strip().split('\n')
        if lines:
            coverage = int(lines[0].split('\t')[2])
            return coverage
    return None

In [7]:
def Position2gene(row, gene2regions):
    df = gene2regions[(gene2regions["chrom"]==row.chrom)
                      &(gene2regions["start"]<=int(row.pos))
                      &(gene2regions["end"]>=int(row.pos))]
    genes = list(df["gene"].values)
    if len(genes)>0:
        return genes[0]
    else:
        return "nan"

In [ ]:
# BAM file directory
bam_folder = "/home/bioit/mlarmuse/Documents/Data/Almac_project/CopenHagen_oligo/tNGS_bams/"

In [ ]:
# Patient ID4
pat_id = "ID4"
num_id = "4"
ID4ids = ["HR41", "HR42", "HR43", "HR44", "HR45", "HR4MLN1", "HR4MLN2", "HR4MLN3"]
ID4_variants = pd.DataFrame()
for f in ID4ids:
    var = SNPandIndel(f+".somaticFilter", chrom2use=chrom2use)
    var["sampleID"] = f
    ID4_variants = pd.concat([ID4_variants, var], axis=0)
    
ID4_df = ID4_variants.groupby(['VC'])['sampleID'].agg(list).reset_index()
ID4_df["multiple"] = ID4_df["sampleID"].str.len()
ID4_df["MLN"]  = [1 if any("MLN" in y for y in x) else 0 for x in ID4_df["sampleID"]]
ID4_df = ID4_df[(ID4_df["multiple"]>1)&(ID4_df["MLN"]==1)]
ID4_df["chrom"] = ID4_df["VC"].str.split("_", expand=True)[0]
ID4_df["pos"] = ID4_df["VC"].str.split("_", expand=True)[1]

ID4_T_bam = ["OT-P-"+pat_id+"-T-"+x+"-KH20191030-C320191031.clip.overlapped.bam" for x in ID4ids]
ID4_N_bam = ["OT-P-"+pat_id+"-N-"+"HR"+num_id+"NT"+"-KH20191030-C320191031.clip.overlapped.bam"]
ID4_T_N_bam = ID4_T_bam+ID4_N_bam

for f in ID4_T_N_bam:
    name = f.split("-")[4]
    ID4_df[name] = ID4_df.apply(GetCoverageAtPos, bam_file=bam_folder+pat_id+"/"+f, axis=1)

col2use = [c for c in ID4_df.columns if "HR4" in c]
ID4_df = ID4_df[ID4_df[col2use].min(axis=1)>=50]

ID4_res = pd.merge(ID4_df, ID4_variants, on=["VC"])

In [ ]:
print(len(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR41"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN1"])),
      len(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR42"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN1"])),
     len(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR43"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN1"])),
     len(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR44"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN1"])),
     len(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR45"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN1"])))

print(len(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR41"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN2"])),
      len(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR42"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN2"])),
     len(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR43"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN2"])),
     len(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR44"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN2"])),
     len(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR45"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN2"])))

print(len(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR41"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN3"])),
      len(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR42"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN3"])),
     len(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR43"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN3"])),
     len(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR44"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN3"])),
     len(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR45"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN3"])))

In [ ]:
print(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR42"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN1"]),
     set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR43"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN1"]))

print(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR42"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN2"]),
     set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR43"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN2"]))

print(set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR42"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN3"]),
     set(ID4_res["VC"][ID4_res["sampleID_y"]=="HR43"]).intersection(
    ID4_res["VC"][ID4_res["sampleID_y"]=="HR4MLN3"]))

In [ ]:
# Patient ID5
pat_id = "ID5"
num_id = "5"
ID5ids = ["HR51", "HR52", "HR53", "HR54", "HR5MLN1", "HR5MLN2", "HR5MLN3"]
ID5_variants = pd.DataFrame()
for f in ID5ids:
    var = SNPandIndel(f+".somaticFilter", chrom2use=chrom2use)
    var["sampleID"] = f
    ID5_variants = pd.concat([ID5_variants, var], axis=0)
    
ID5_df = ID5_variants.groupby(['VC'])['sampleID'].agg(list).reset_index()
ID5_df["multiple"] = ID5_df["sampleID"].str.len()
ID5_df["MLN"]  = [1 if any("MLN" in y for y in x) else 0 for x in ID5_df["sampleID"]]
ID5_df = ID5_df[(ID5_df["multiple"]>1)&(ID5_df["MLN"]==1)]
ID5_df["chrom"] = ID5_df["VC"].str.split("_", expand=True)[0]
ID5_df["pos"] = ID5_df["VC"].str.split("_", expand=True)[1]

ID5_T_bam = ["OT-P-"+pat_id+"-T-"+x+"-KH20191030-C320191031.clip.overlapped.bam" for x in ID5ids]
ID5_N_bam = ["OT-P-"+pat_id+"-N-"+"HR"+num_id+"NT"+"-KH20191030-C320191031.clip.overlapped.bam"]
ID5_T_N_bam = ID5_T_bam+ID5_N_bam

for f in ID5_T_N_bam:
    name = f.split("-")[4]
    ID5_df[name] = ID5_df.apply(GetCoverageAtPos, bam_file=bam_folder+pat_id+"/"+f, axis=1)

col2use = [c for c in ID5_df.columns if "HR5" in c]
ID5_df = ID5_df[ID5_df[col2use].min(axis=1)>=50]

ID5_res = pd.merge(ID5_df, ID5_variants, on=["VC"])

In [ ]:
print(len(set(ID5_res["VC"][ID5_res["sampleID_y"]=="HR51"]).intersection(
    ID5_res["VC"][ID5_res["sampleID_y"]=="HR5MLN1"])),
      len(set(ID5_res["VC"][ID5_res["sampleID_y"]=="HR52"]).intersection(
    ID5_res["VC"][ID5_res["sampleID_y"]=="HR5MLN1"])),
     len(set(ID5_res["VC"][ID5_res["sampleID_y"]=="HR53"]).intersection(
    ID5_res["VC"][ID5_res["sampleID_y"]=="HR5MLN1"])),
     len(set(ID5_res["VC"][ID5_res["sampleID_y"]=="HR54"]).intersection(
    ID5_res["VC"][ID5_res["sampleID_y"]=="HR5MLN1"])))

print(len(set(ID5_res["VC"][ID5_res["sampleID_y"]=="HR51"]).intersection(
    ID5_res["VC"][ID5_res["sampleID_y"]=="HR5MLN2"])),
      len(set(ID5_res["VC"][ID5_res["sampleID_y"]=="HR52"]).intersection(
    ID5_res["VC"][ID5_res["sampleID_y"]=="HR5MLN2"])),
     len(set(ID5_res["VC"][ID5_res["sampleID_y"]=="HR53"]).intersection(
    ID5_res["VC"][ID5_res["sampleID_y"]=="HR5MLN2"])),
     len(set(ID5_res["VC"][ID5_res["sampleID_y"]=="HR54"]).intersection(
    ID5_res["VC"][ID5_res["sampleID_y"]=="HR5MLN2"])))

print(len(set(ID5_res["VC"][ID5_res["sampleID_y"]=="HR51"]).intersection(
    ID5_res["VC"][ID5_res["sampleID_y"]=="HR5MLN3"])),
      len(set(ID5_res["VC"][ID5_res["sampleID_y"]=="HR52"]).intersection(
    ID5_res["VC"][ID5_res["sampleID_y"]=="HR5MLN3"])),
     len(set(ID5_res["VC"][ID5_res["sampleID_y"]=="HR53"]).intersection(
    ID5_res["VC"][ID5_res["sampleID_y"]=="HR5MLN3"])),
     len(set(ID5_res["VC"][ID5_res["sampleID_y"]=="HR54"]).intersection(
    ID5_res["VC"][ID5_res["sampleID_y"]=="HR5MLN3"])))

In [ ]:
# Patient ID6
pat_id = "ID6"
num_id = "6"
ID6ids = ["HR61", "HR62", "HR63", "HR64", "HR6MLN1", "HR6MLN2", "HR6MLN3"]
ID6_variants = pd.DataFrame()
for f in ID6ids:
    var = SNPandIndel(f+".somaticFilter", chrom2use=chrom2use)
    var["sampleID"] = f
    ID6_variants = pd.concat([ID6_variants, var], axis=0)
    
ID6_df = ID6_variants.groupby(['VC'])['sampleID'].agg(list).reset_index()
ID6_df["multiple"] = ID6_df["sampleID"].str.len()
ID6_df["MLN"]  = [1 if any("MLN" in y for y in x) else 0 for x in ID6_df["sampleID"]]
ID6_df = ID6_df[(ID6_df["multiple"]>1)&(ID6_df["MLN"]==1)]
ID6_df["chrom"] = ID6_df["VC"].str.split("_", expand=True)[0]
ID6_df["pos"] = ID6_df["VC"].str.split("_", expand=True)[1]

ID6_T_bam = ["OT-P-"+pat_id+"-T-"+x+"-KH20191030-C320191031.clip.overlapped.bam" for x in ID6ids]
ID6_N_bam = ["OT-P-"+pat_id+"-N-"+"HR"+num_id+"NT"+"-KH20191030-C320191031.clip.overlapped.bam"]
ID6_T_N_bam = ID6_T_bam+ID6_N_bam

for f in ID6_T_N_bam:
    name = f.split("-")[4]
    ID6_df[name] = ID6_df.apply(GetCoverageAtPos, bam_file=bam_folder+pat_id+"/"+f, axis=1)

col2use = [c for c in ID6_df.columns if "HR6" in c]
ID6_df = ID6_df[ID6_df[col2use].min(axis=1)>=50]

ID6_res = pd.merge(ID6_df, ID6_variants, on=["VC"])

In [ ]:
print(len(set(ID6_res["VC"][ID6_res["sampleID_y"]=="HR61"]).intersection(
    ID6_res["VC"][ID6_res["sampleID_y"]=="HR6MLN1"])),
      len(set(ID6_res["VC"][ID6_res["sampleID_y"]=="HR62"]).intersection(
    ID6_res["VC"][ID6_res["sampleID_y"]=="HR6MLN1"])),
     len(set(ID6_res["VC"][ID6_res["sampleID_y"]=="HR63"]).intersection(
    ID6_res["VC"][ID6_res["sampleID_y"]=="HR6MLN1"])),
     len(set(ID6_res["VC"][ID6_res["sampleID_y"]=="HR64"]).intersection(
    ID6_res["VC"][ID6_res["sampleID_y"]=="HR6MLN1"])),
     len(set(ID6_res["VC"][ID6_res["sampleID_y"]=="HR65"]).intersection(
    ID6_res["VC"][ID6_res["sampleID_y"]=="HR6MLN1"])))

print(len(set(ID6_res["VC"][ID6_res["sampleID_y"]=="HR61"]).intersection(
    ID6_res["VC"][ID6_res["sampleID_y"]=="HR6MLN2"])),
      len(set(ID6_res["VC"][ID6_res["sampleID_y"]=="HR62"]).intersection(
    ID6_res["VC"][ID6_res["sampleID_y"]=="HR6MLN2"])),
     len(set(ID6_res["VC"][ID6_res["sampleID_y"]=="HR63"]).intersection(
    ID6_res["VC"][ID6_res["sampleID_y"]=="HR6MLN2"])),
     len(set(ID6_res["VC"][ID6_res["sampleID_y"]=="HR64"]).intersection(
    ID6_res["VC"][ID6_res["sampleID_y"]=="HR6MLN2"])),
     len(set(ID6_res["VC"][ID6_res["sampleID_y"]=="HR65"]).intersection(
    ID6_res["VC"][ID6_res["sampleID_y"]=="HR6MLN2"])))

print(len(set(ID6_res["VC"][ID6_res["sampleID_y"]=="HR61"]).intersection(
    ID6_res["VC"][ID6_res["sampleID_y"]=="HR6MLN3"])),
      len(set(ID6_res["VC"][ID6_res["sampleID_y"]=="HR62"]).intersection(
    ID6_res["VC"][ID6_res["sampleID_y"]=="HR6MLN3"])),
     len(set(ID6_res["VC"][ID6_res["sampleID_y"]=="HR63"]).intersection(
    ID6_res["VC"][ID6_res["sampleID_y"]=="HR6MLN3"])),
     len(set(ID6_res["VC"][ID6_res["sampleID_y"]=="HR64"]).intersection(
    ID6_res["VC"][ID6_res["sampleID_y"]=="HR6MLN3"])),
     len(set(ID6_res["VC"][ID6_res["sampleID_y"]=="HR65"]).intersection(
    ID6_res["VC"][ID6_res["sampleID_y"]=="HR6MLN3"])))

In [ ]:
# Patient ID9
pat_id = "ID9"
num_id = "9"
ID9ids = ["HR91", "HR92", "HR93", "HR94", "HR95", "HR9MLN1", "HR9MLN2"]
ID9_variants = pd.DataFrame()
for f in ID9ids:
    var = SNPandIndel(f+".somaticFilter", chrom2use=chrom2use)
    var["sampleID"] = f
    ID9_variants = pd.concat([ID9_variants, var], axis=0)
    
ID9_df = ID9_variants.groupby(['VC'])['sampleID'].agg(list).reset_index()
ID9_df["multiple"] = ID9_df["sampleID"].str.len()
ID9_df["MLN"]  = [1 if any("MLN" in y for y in x) else 0 for x in ID9_df["sampleID"]]
ID9_df = ID9_df[(ID9_df["multiple"]>1)&(ID9_df["MLN"]==1)]
ID9_df["chrom"] = ID9_df["VC"].str.split("_", expand=True)[0]
ID9_df["pos"] = ID9_df["VC"].str.split("_", expand=True)[1]

ID9_T_bam = ["OT-P-"+pat_id+"-T-"+x+"-KH20191030-C320191031.clip.overlapped.bam" for x in ID9ids]
ID9_N_bam = ["OT-P-"+pat_id+"-N-"+"HR"+num_id+"NT"+"-KH20191030-C320191031.clip.overlapped.bam"]
ID9_T_N_bam = ID9_T_bam+ID9_N_bam

for f in ID9_T_N_bam:
    name = f.split("-")[4]
    ID9_df[name] = ID9_df.apply(GetCoverageAtPos, bam_file=bam_folder+pat_id+"/"+f, axis=1)
    
col2use = [c for c in ID9_df.columns if "HR9" in c]
ID9_df = ID9_df[ID9_df[col2use].min(axis=1)>=50]

ID9_res = pd.merge(ID9_df, ID9_variants, on=["VC"])

In [ ]:
print(len(set(ID9_res["VC"][ID9_res["sampleID_y"]=="HR91"]).intersection(
    ID9_res["VC"][ID9_res["sampleID_y"]=="HR9MLN1"])),
      len(set(ID9_res["VC"][ID9_res["sampleID_y"]=="HR92"]).intersection(
    ID9_res["VC"][ID9_res["sampleID_y"]=="HR9MLN1"])),
     len(set(ID9_res["VC"][ID9_res["sampleID_y"]=="HR93"]).intersection(
    ID9_res["VC"][ID9_res["sampleID_y"]=="HR9MLN1"])),
     len(set(ID9_res["VC"][ID9_res["sampleID_y"]=="HR94"]).intersection(
    ID9_res["VC"][ID9_res["sampleID_y"]=="HR9MLN1"])),
     len(set(ID9_res["VC"][ID9_res["sampleID_y"]=="HR95"]).intersection(
    ID9_res["VC"][ID9_res["sampleID_y"]=="HR9MLN1"])))

print(len(set(ID9_res["VC"][ID9_res["sampleID_y"]=="HR91"]).intersection(
    ID9_res["VC"][ID9_res["sampleID_y"]=="HR9MLN2"])),
      len(set(ID9_res["VC"][ID9_res["sampleID_y"]=="HR92"]).intersection(
    ID9_res["VC"][ID9_res["sampleID_y"]=="HR9MLN2"])),
     len(set(ID9_res["VC"][ID9_res["sampleID_y"]=="HR93"]).intersection(
    ID9_res["VC"][ID9_res["sampleID_y"]=="HR9MLN2"])),
     len(set(ID9_res["VC"][ID9_res["sampleID_y"]=="HR94"]).intersection(
    ID9_res["VC"][ID9_res["sampleID_y"]=="HR9MLN2"])),
     len(set(ID9_res["VC"][ID9_res["sampleID_y"]=="HR95"]).intersection(
    ID9_res["VC"][ID9_res["sampleID_y"]=="HR9MLN2"])))

In [ ]:
# Patient ID10
pat_id = "ID10"
num_id = "10"
ID10ids = ["HR101", "HR102", "HR103", "HR104", "HR105", "HR106", "HR10MLN1", "HR10MLN2"]
ID10_variants = pd.DataFrame()
for f in ID10ids:
    var = SNPandIndel(f+".somaticFilter", chrom2use=chrom2use)
    var["sampleID"] = f
    ID10_variants = pd.concat([ID10_variants, var], axis=0)
    
ID10_df = ID10_variants.groupby(['VC'])['sampleID'].agg(list).reset_index()
ID10_df["multiple"] = ID10_df["sampleID"].str.len()
ID10_df["MLN"]  = [1 if any("MLN" in y for y in x) else 0 for x in ID10_df["sampleID"]]
ID10_df = ID10_df[(ID10_df["multiple"]>1)&(ID10_df["MLN"]==1)]
ID10_df["chrom"] = ID10_df["VC"].str.split("_", expand=True)[0]
ID10_df["pos"] = ID10_df["VC"].str.split("_", expand=True)[1]

ID10_T_bam = ["OT-P-"+pat_id+"-T-"+x+"-KH20191030-C320191031.clip.overlapped.bam" for x in ID10ids]
ID10_N_bam = ["OT-P-"+pat_id+"-N-"+"HR"+num_id+"NT"+"-KH20191030-C320191031.clip.overlapped.bam"]
ID10_T_N_bam = ID10_T_bam+ID10_N_bam

for f in ID10_T_N_bam:
    name = f.split("-")[4]
    ID10_df[name] = ID10_df.apply(GetCoverageAtPos, bam_file=bam_folder+pat_id+"/"+f, axis=1)

col2use = [c for c in ID10_df.columns if "HR10" in c]
ID10_df = ID10_df[ID10_df[col2use].min(axis=1)>=50]

ID10_res = pd.merge(ID10_df, ID10_variants, on=["VC"])

In [ ]:
print(len(set(ID10_res["VC"][ID10_res["sampleID_y"]=="HR101"]).intersection(
    ID10_res["VC"][ID10_res["sampleID_y"]=="HR10MLN1"])),
      len(set(ID10_res["VC"][ID10_res["sampleID_y"]=="HR102"]).intersection(
    ID10_res["VC"][ID10_res["sampleID_y"]=="HR10MLN1"])),
     len(set(ID10_res["VC"][ID10_res["sampleID_y"]=="HR103"]).intersection(
    ID10_res["VC"][ID10_res["sampleID_y"]=="HR10MLN1"])),
     len(set(ID10_res["VC"][ID10_res["sampleID_y"]=="HR104"]).intersection(
    ID10_res["VC"][ID10_res["sampleID_y"]=="HR10MLN1"])),
     len(set(ID10_res["VC"][ID10_res["sampleID_y"]=="HR105"]).intersection(
    ID10_res["VC"][ID10_res["sampleID_y"]=="HR10MLN1"])),
      len(set(ID10_res["VC"][ID10_res["sampleID_y"]=="HR106"]).intersection(
    ID10_res["VC"][ID10_res["sampleID_y"]=="HR10MLN1"])))

print(len(set(ID10_res["VC"][ID10_res["sampleID_y"]=="HR101"]).intersection(
    ID10_res["VC"][ID10_res["sampleID_y"]=="HR10MLN2"])),
      len(set(ID10_res["VC"][ID10_res["sampleID_y"]=="HR102"]).intersection(
    ID10_res["VC"][ID10_res["sampleID_y"]=="HR10MLN2"])),
     len(set(ID10_res["VC"][ID10_res["sampleID_y"]=="HR103"]).intersection(
    ID10_res["VC"][ID10_res["sampleID_y"]=="HR10MLN2"])),
     len(set(ID10_res["VC"][ID10_res["sampleID_y"]=="HR104"]).intersection(
    ID10_res["VC"][ID10_res["sampleID_y"]=="HR10MLN2"])),
     len(set(ID10_res["VC"][ID10_res["sampleID_y"]=="HR105"]).intersection(
    ID10_res["VC"][ID10_res["sampleID_y"]=="HR10MLN2"])),
     len(set(ID10_res["VC"][ID10_res["sampleID_y"]=="HR106"]).intersection(
    ID10_res["VC"][ID10_res["sampleID_y"]=="HR10MLN2"])))


In [ ]:
# Gene to regions (hg19) - coding regions
hgtable = pd.read_csv("/home/bioit/ldschaet/Desktop/FASTQ_tNGS_LA/gene_table.gz", sep="\t")
hgtable = hgtable[['name2', "chrom", "txStart", "txEnd"]].drop_duplicates()
hgtable.columns = ["gene", "chrom", "start", "end"]
hgtable["chrom"] = hgtable["chrom"].str[3:]
hgtable["size"] = hgtable["end"]-hgtable["start"]
hgtable

In [ ]:
ID4_res["genes"] = ID4_res.apply(Position2gene, gene2regions=hgtable, axis=1)
ID5_res["genes"] = ID5_res.apply(Position2gene, gene2regions=hgtable, axis=1)
ID6_res["genes"] = ID6_res.apply(Position2gene, gene2regions=hgtable, axis=1)
ID9_res["genes"] = ID9_res.apply(Position2gene, gene2regions=hgtable, axis=1)
ID10_res["genes"] = ID10_res.apply(Position2gene, gene2regions=hgtable, axis=1)

ID4_res = ID4_res.drop('sampleID_x', axis=1)
ID5_res = ID5_res.drop('sampleID_x', axis=1)
ID6_res = ID6_res.drop('sampleID_x', axis=1)
ID9_res = ID9_res.drop('sampleID_x', axis=1)
ID10_res = ID10_res.drop('sampleID_x', axis=1)

In [ ]:
var2use = []
for var in set(ID4_res["VC"]):
    if max(ID4_res[ID4_res["VC"]==var]["T_VAF"])>0.05:
           var2use.append(var)
ID4_res[["VC", "multiple", "T_VAF", "T_ALT", "sampleID_y", "genes"]][
    (ID4_res["VC"].isin(var2use))
].sort_values(by=["multiple", "VC"], ascending=False)

In [ ]:
var2use = []
for var in set(ID5_res["VC"]):
    if max(ID5_res[ID5_res["VC"]==var]["T_VAF"])>0.05:
           var2use.append(var)
ID5_res[["VC", "multiple", "T_VAF", "T_ALT", "sampleID_y", "genes"]][
    (ID5_res["VC"].isin(var2use))
].sort_values(by=["multiple", "VC"], ascending=False)

In [ ]:
var2use = []
for var in set(ID6_res["VC"]):
    if max(ID6_res[ID6_res["VC"]==var]["T_VAF"])>0.05:
           var2use.append(var)
ID6_res[["VC", "multiple", "T_VAF", "T_ALT", "sampleID_y", "genes"]][
    ID6_res["VC"].isin(var2use)
].sort_values(by=["multiple", "VC"], ascending=False)

In [ ]:
var2use = []
for var in set(ID9_res["VC"]):
    if max(ID9_res[ID9_res["VC"]==var]["T_VAF"])>0.05:
           var2use.append(var)
ID9_res[["VC", "multiple", "T_VAF", "T_ALT", "sampleID_y", "genes"]][
    ID9_res["VC"].isin(var2use)
].sort_values(by=["multiple", "VC"], ascending=False)

In [ ]:
var2use = []
for var in set(ID10_res["VC"]):
    if max(ID10_res[ID10_res["VC"]==var]["T_VAF"])>0.05:
           var2use.append(var)
ID10_res[["VC", "multiple", "T_VAF", "T_ALT", "sampleID_y", "genes"]][
    ID10_res["VC"].isin(var2use)
].sort_values(by=["multiple", "VC"], ascending=False)

In [ ]:
ID6_res[["VC", "multiple", "T_VAF", "T_ALT", "sampleID_y", "genes"]].sort_values(by=["multiple", "VC"], ascending=False)

In [ ]:
ID9_res[["VC", "multiple", "T_VAF", "T_ALT", "sampleID_y", "genes"]].sort_values(by=["multiple", "VC"], ascending=False)

In [ ]:
ID10_res[["VC", "multiple", "T_VAF", "T_ALT", "sampleID_y", "genes"]].sort_values(by=["multiple", "VC"], ascending=False)

In [ ]:
ID4_res.to_csv("/home/bioit/ldschaet/Desktop/FASTQ_tNGS_LA/Results/ID4_res.csv", index=False, header=True)
ID5_res.to_csv("/home/bioit/ldschaet/Desktop/FASTQ_tNGS_LA/Results/ID5_res.csv", index=False, header=True)
ID6_res.to_csv("/home/bioit/ldschaet/Desktop/FASTQ_tNGS_LA/Results/ID6_res.csv", index=False, header=True)
ID9_res.to_csv("/home/bioit/ldschaet/Desktop/FASTQ_tNGS_LA/Results/ID9_res.csv", index=False, header=True)
ID10_res.to_csv("/home/bioit/ldschaet/Desktop/FASTQ_tNGS_LA/Results/ID10_res.csv", index=False, header=True)

In [ ]:
ID4_res

In [ ]:
ID5_res[(ID5_res["type"]=="snp")]["sampleID_y"].value_counts()

In [ ]:
ID6_res[(ID6_res["type"]=="snp")]["sampleID_y"].value_counts()

In [ ]:
ID9_res[(ID9_res["type"]=="snp")]["sampleID_y"].value_counts()

In [ ]:
ID10_res[(ID10_res["type"]=="snp")&((ID10_res["T_ALT"]>5))]["sampleID_y"].value_counts()

In [ ]:
ID9_res[(ID9_res["type"]=="snp")&(ID9_res["T_ALT"]>5)]["sampleID_y"].value_counts()

In [ ]:
ID9_res[(ID9_res["type"]=="snp")&(ID9_res["T_ALT"]>5)]["sampleID_y"].value_counts()